# File Chunking
Partitioning large documents into smaller chunks can help you stay under the maximum token input limits of embedding models. For example, the maximum length of input text for the [Azure OpenAI](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/embeddings) embedding models is 8,191 tokens. Given that each token is around four characters of text for common OpenAI models, this maximum limit is equivalent to around 6,000 words of text. If you're using these models to generate embeddings, it's critical that the input text stays under the limit. Partitioning your content into chunks ensures that your data can be processed by the embedding models used to populate vector stores and text-to-vector query conversions.

This notebook walks through the process used in the [chat-with-your-data-solution-accelerator](https://github.com/Azure-Samples/chat-with-your-data-solution-accelerator/tree/main) repo of analysing file content in [AI Document Intelligence](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-4.0.0), then breaking it into smaller overlapping chunks using LangChain. These chunks can then be vectorised and stored in a service such as [Azure AI Search](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search) which provides secure information retrieval at scale over user-owned content in traditional and generative AI search applications. 

> NOTE: Before proceeding, you need to complete the steps in the first `00_Setup` notebook.

### Install dependencies
First we install the dependencies required in ths notebook.

In [6]:
# Install dependencies
# Fixing the import issue of dotenv lib
#%pip install dotenv
%pip install python-dotenv
%pip install langchain
%pip install azure.ai.formrecognizer
%pip install tiktoken

   ---------------------------------------- 0.0/799.0 kB ? eta -:--:--
   --------------------------------------- 799.0/799.0 kB 11.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Load credentials
Next we load the environment variables needed by the following cells from the `.env` file.

In [1]:
# Load credentials
import os
from dotenv import load_dotenv
# fixing the env file
load_dotenv()

# fixing the code error
#print(f"blob endpoint={os.getenv("BLOB_ENDPOINT")}")
print(f"blob endpoint={os.getenv('BLOB_ENDPOINT')}")

blob endpoint=https://ukhosampledocs.blob.core.windows.net/


### Set execution parameters
Set the name of the file you uploaded to the blob container (that we are going to chunk) and the container name. The filename is joined with the blob endpoint, container name, and SAS token to create a url to the file. You can also adjust the `chunk_size` and `chunk_overlap` to see their effect on the output later, or leave the defaults.

In [2]:
## The name of the file you uploaded to your Blob Storage Datasource
# file_name="employee_handbook.pdf"
file_name="Cleowent Agreement with Tidal Licence - 03.01.2012.pdf"

# Name of the container in your Blob Storage Datasource (in credentials.env)
# blob_container_name = "data"
blob_container_name = os.getenv("BLOB_CONTAINER_NAME")

file_sas=os.getenv("BLOB_SAS")
file_url=os.getenv("BLOB_ENDPOINT") + blob_container_name + "/" + file_name + "?" + file_sas

print(f"file url={file_url}")

# You can play with these parameters to see how they affect the output
my_chunk_size = 500
my_chunk_overlap = 100

file url=https://ukhosampledocs.blob.core.windows.net/docs/Cleowent Agreement with Tidal Licence - 03.01.2012.pdf?sp=r&st=2024-09-06T16:52:22Z&se=2024-09-07T00:52:22Z&spr=https&sv=2022-11-02&sr=b&sig=3hRdoZb5z4ZDo%2F%2FyTFNqwYKHShj5VQfuM62dnhfamAw%3D


### Read the file in with Document Intelligence
[Azure AI Document Intelligence](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-4.0.0) is a cloud-based Azure AI service that enables you to build intelligent document processing solutions. Massive amounts of data, spanning a wide variety of data types, are stored in forms and documents. Document Intelligence enables you to effectively manage the velocity at which data is collected and processed and is key to improved operations, informed data-driven decisions, and enlightened innovation.

This cell creates a `DocumentAnalysisClient` object using the settings in the credentials file. It then defines some helper functions to do things like converting tables to HTML.

The main part of the code calls the *DocumentAnalysisClient* `begin_analyze_document_from_url` method to analyse the file using the pre-defined `layout` [model](https://learn.microsoft.com/en-gb/azure/ai-services/document-intelligence/concept-layout?view=doc-intel-4.0.0&tabs=sample-code). The output of the DocumentAnalysisClient call is then parsed and converted into HTML formatted *pages*, while keeping track of paragraph and table roles and positions.

In [3]:
#code\backend\batch\utilities\helpers\azure_form_recognizer_helper.py

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import html
import traceback


#print(f"form recognizer={os.getenv("FORM_RECOGNIZER_ENDPOINT")}")
print(f"form recognizer={os.getenv('FORM_RECOGNIZER_ENDPOINT')}")

document_analysis_client = DocumentAnalysisClient(
    endpoint=os.getenv("FORM_RECOGNIZER_ENDPOINT"), 
    credential=AzureKeyCredential(os.getenv("FORM_RECOGNIZER_KEY"))
)

form_recognizer_role_to_html = {
    "title": "h1",
    "sectionHeading": "h2",
    "pageHeader": None,
    "pageFooter": None,
    "paragraph": "p",
}

def _table_to_html(table):
    table_html = "<table>"
    rows = [
        sorted(
            [cell for cell in table.cells if cell.row_index == i],
            key=lambda cell: cell.column_index,
        )
        for i in range(table.row_count)
    ]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = (
                "th"
                if (cell.kind == "columnHeader" or cell.kind == "rowHeader")
                else "td"
            )
            cell_spans = ""
            if cell.column_span > 1:
                cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1:
                cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html += "</tr>"
    table_html += "</table>"
    return table_html


offset = 0
page_map = []
model_id = "prebuilt-layout"

try:
    poller = document_analysis_client.begin_analyze_document_from_url(
        model_id, document_url=file_url
    )
    form_recognizer_results = poller.result()

    roles_start = {}
    roles_end = {}
    for paragraph in form_recognizer_results.paragraphs:
        # if paragraph.role!=None:
        para_start = paragraph.spans[0].offset
        para_end = paragraph.spans[0].offset + paragraph.spans[0].length
        roles_start[para_start] = (
            paragraph.role if paragraph.role is not None else "paragraph"
        )
        roles_end[para_end] = (
            paragraph.role if paragraph.role is not None else "paragraph"
        )

    for page_num, page in enumerate(form_recognizer_results.pages):
        tables_on_page = [
            table
            for table in form_recognizer_results.tables
            if table.bounding_regions[0].page_number == page_num + 1
        ]

        # (if using layout) mark all positions of the table spans in the page
        page_offset = page.spans[0].offset
        page_length = page.spans[0].length
        table_chars = [-1] * page_length
        for table_id, table in enumerate(tables_on_page):
            for span in table.spans:
                # replace all table spans with "table_id" in table_chars array
                for i in range(span.length):
                    idx = span.offset - page_offset + i
                    if idx >= 0 and idx < page_length:
                        table_chars[idx] = table_id

        # build page text by replacing charcters in table spans with table html and replace the characters corresponding to headers with html headers, if using layout
        page_text = ""
        added_tables = set()
        for idx, table_id in enumerate(table_chars):
            if table_id == -1:
                position = page_offset + idx
                if position in roles_start.keys():
                    role = roles_start[position]
                    html_role = form_recognizer_role_to_html.get(role)
                    if html_role is not None:
                        page_text += f"<{html_role}>"
                if position in roles_end.keys():
                    role = roles_end[position]
                    html_role = form_recognizer_role_to_html.get(role)
                    if html_role is not None:
                        page_text += f"</{html_role}>"

                page_text += form_recognizer_results.content[page_offset + idx]

            elif table_id not in added_tables:
                page_text += _table_to_html(tables_on_page[table_id])
                added_tables.add(table_id)

        page_text += " "
        page_map.append(
            {"page_number": page_num, "offset": offset, "page_text": page_text}
        )
        offset += len(page_text)

    print(f"page map={page_map}")

except Exception as e:
    raise ValueError(f"Error: {traceback.format_exc()}. Error: {e}")

form recognizer=https://ukho-ai-doc-intelligence.cognitiveservices.azure.com/
page map=[{'page_number': 0, 'offset': 0, 'page_text': '<h1>AGREEMENT</h1>\n<p>BETWEEN</p>\n<p>THE UNITED KINGDOM HYDROGRAPHIC OFFICE</p>\n<p>AND</p>\n<p>CLEOWENT HARBOUR COMMISSION</p>\nVersion 15 November 2011 '}, {'page_number': 1, 'offset': 151, 'page_text': "<h1>AGREEMENT BETWEEN THE UNITED KINGDOM HYDROGRAPHIC OFFICE AND CLEOWENT HARBOUR COMMISSION</h1>\n<h2>PURPOSE OF THIS AGREEMENT</h2>\n<p>1. The United Kingdom Hydrographic Office on behalf of the Secretary of State for Defence (the UKHO) and Cleowent Harbour Commission (CHC) (together the Parties) have decided to enter a non-exclusive Agreement (this Agreement), for their mutual benefit in assuring the supply of hydrographic surveys, other data, and related information by CHC to the UKHO for inclusion in charts and other Products, and the supply of the UKHO's Products and related information to CHC for use in its work.</p>\n<p>2. This Agreement form

### Parse into a document object
The following cell takes the *page_map* created in the previous cell and builds a list of `SourceDocument` objects, one for each page in the page_map, which are used to simplify the next phase of processing. 

In [5]:
#code\backend\batch\utilities\document_loading\layout.py

from SourceDocument import SourceDocument

pages_content = page_map

documents = [
    SourceDocument(
        content=page["page_text"],
        source=file_url,
        offset=page["offset"],
        page_number=page["page_number"],
    )
    for page in pages_content
    ]

### Chunk the document
The following cell splits the contents of the entire document into chunks based on the parameters set in the earlier cell using [LangChain text splitters](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/). A final list `chunked_documents` contains `SourceDocument` objects with the specified chunk_size and chunk_overlap. 

In [6]:
#code\backend\batch\utilities\document_chunking\layout.py

from typing import List
from langchain.text_splitter import MarkdownTextSplitter

# Combine all pages into a single document
full_document_content = "".join(
    list(map(lambda document: document.content, documents))
)

# Split the document into chunks
document_url = documents[0].source
splitter = MarkdownTextSplitter.from_tiktoken_encoder(
    chunk_size=my_chunk_size, chunk_overlap=my_chunk_overlap
)
chunked_content_list = splitter.split_text(full_document_content)

# Create a list of SourceDocuments from the chunked content
chunked_documents = []
chunk_offset = 0
for idx, chunked_content in enumerate(chunked_content_list):
    chunked_documents.append(
        SourceDocument.from_metadata(
            content=chunked_content,
            document_url=document_url,
            metadata={"offset": chunk_offset},
            idx=idx,
        )
    )

    # Print each chunk
    print(f"offset={chunk_offset}")
    print(f"chunk={idx}: {chunked_content}\n")

    chunk_offset += len(chunked_content)

#"chunked_documents" variable contains the list of documents split into chunks

offset=0
chunk=0: <h1>AGREEMENT</h1>
<p>BETWEEN</p>
<p>THE UNITED KINGDOM HYDROGRAPHIC OFFICE</p>
<p>AND</p>
<p>CLEOWENT HARBOUR COMMISSION</p>
Version 15 November 2011 <h1>AGREEMENT BETWEEN THE UNITED KINGDOM HYDROGRAPHIC OFFICE AND CLEOWENT HARBOUR COMMISSION</h1>
<h2>PURPOSE OF THIS AGREEMENT</h2>
<p>1. The United Kingdom Hydrographic Office on behalf of the Secretary of State for Defence (the UKHO) and Cleowent Harbour Commission (CHC) (together the Parties) have decided to enter a non-exclusive Agreement (this Agreement), for their mutual benefit in assuring the supply of hydrographic surveys, other data, and related information by CHC to the UKHO for inclusion in charts and other Products, and the supply of the UKHO's Products and related information to CHC for use in its work.</p>
<p>2. This Agreement formalises the exchanges between the Parties and seeks to encourage cooperation between them.</p>
<p>DEFINITIONS</p>
<p>3. Defined terms are set out at Appendix 1.</p>
<p>EXCHANGE 